In [1]:
# Following is the implementation without using ordereddict.
class Node:
    def __init__(self, key, val):
        self.val = val
        self.key = key
        self.next = None
        self.prev = None
        
class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
    
    def insert(self, node):
        if self.head is None:
            self.head = node
        else:
            self.tail.next = node
            node.prev = self.tail
        self.tail = node
            
    def delete(self, node):
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next
            
        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev

class LRUCache:
    def __init__(self, capacity):
        self.list = LinkedList()
        self.dict = {}
        self.capacity = capacity
        
    def _insert(self, key, val):
        node = Node(key, val)
        self.list.insert(node)
        self.dict[key] = node

    def get(self, key):
        if key in self.dict:
            val = self.dict[key].val
            self.list.delete(self.dict[key])
            self._insert(key, val)
            return val
        return -1

    def set(self, key, val):
        if key in self.dict:
            self.list.delete(self.dict[key])
        elif len(self.dict) == self.capacity:
            del self.dict[self.list.head.key]
            self.list.delete(self.list.head)
        self._insert(key, val)

lRUCache =  LRUCache(2)
lRUCache.set(1, 1) # cache is {1=1}
lRUCache.set(2, 2) # cache is {1=1, 2=2}

print(lRUCache.get(1))    # return 1
lRUCache.set(3, 3) # LRU key was 2, evicts key 2, cache is {1=1, 3=3}
print(lRUCache.get(2) )   # returns -1 (not found)
lRUCache.set(4, 4) # LRU key was 1, evicts key 1, cache is {4=4, 3=3}
print(lRUCache.get(1))   # return -1 (not found)
print(lRUCache.get(3))    # return 3
print(lRUCache.get(4))   # return 4

1
-1
-1
3
4


In [72]:
from enum import Enum

class VehicleType(Enum):
    CAR = 1
    BIKE = 2
    BUS = 3

class Vehicle:
    def __init__(self, licensePlate, name, type_of_vehicle):
        self.licensePlate = licensePlate
        self.name = name
        self.type_of_vehicle = type_of_vehicle
        self.slot = None

    def getType(self):
        return self.type_of_vehicle



    def removeSlot(self):
        self.slot = None

## Car class inherited from Vehicle class for license plate, company name and their type
class Car(Vehicle):
    def __init__(self, licensePlate, companyName):
        Vehicle.__init__(self, licensePlate, companyName, VehicleType.CAR)
class Bike(Vehicle):
    def __init__(self, licensePlate, companyName):
        Vehicle.__init__(self, licensePlate, companyName, VehicleType.BIKE)
class Bus(Vehicle):
    def __init__(self, licensePlate, companyName):
        Vehicle.__init__(self, licensePlate, companyName, VehicleType.BUS)

class Slots:
    def __init__(self, spotNumber, slot_code):
        self.spotNumber = spotNumber
        self.vehicle = None
        self.slot_code = slot_code

    def park(self, vehicle):
        self.vehicle = vehicle
    
    def removeVehicle(self):
        self.vehicle.removeSlot()
        self.vehicle = None

class Levels:
    def __init__(self, floorNumber, no_of_slots, level):
        self.floorNumber = floorNumber
        self.availableSpots = []
        self.availableSpotsDict = {}
        self.level = level

        for slot_no in range(no_of_slots):
            self.availableSpots.append(slot_no)
            slot_code = str(self.level) + "_" + str(slot_no)
            self.availableSpotsDict[slot_no] = Slots(slot_no,slot_code)

    def park(self, vehicle):
        spot_no = self.availableSpots.pop(0)
        slot = self.availableSpotsDict[spot_no]
        slot.park(vehicle)
        vehicle.parkVehicle(slot)

    def remove(self, slot):
        self.availableSpots.append(int(slot))
        self.availableSpotsDict[slot].removeVehicle()


class ParkingLot:
    def __init__(self, no_of_floor, no_of_slot):
        self.levelsDict = {}
        for floor_no in range(no_of_floor):
            self.levelsDict[floor_no] = Levels(floor_no, no_of_slot,floor_no)

    # O( Log(n) )
    def parkVehicle(self, vehicle):
        for level in self.levelsDict.values():
            if len(level.availableSpots) > 0:
                level.park(vehicle)
                break

    # O(1)
    def leaveOperation(self, vehicle):
        level, slot_no = vehicle.slot.slot_code.split("_")
        self.levelsDict[int(level)].remove(int(slot_no))

parkingLotObj = ParkingLot(2, 10)
car_1 = Car(10, "Driver 1")
parkingLotObj.parkVehicle(car_1)
parkingLotObj.leaveOperation(car_1)

In [2]:
from abc import abstractmethod
from enum import Enum

class File:
    def __init__(self, name,size, is_file):
        self.name = name
        self.is_file = is_file
        self.size = size
        self.content = ""
        self.parent = None
        self.children = {}

    def add_children(self, children):
        for f in children:
            f.parent = self
            self.children[f.name] = f
            
    def edit_content(self,content):
        if self.is_file:
            self.content = content
            print("Update Message Successful")
        else:
            print("Invalid Command")

class Operator(Enum):
    Equal, GreaterThan, GreaterThanEqual, LessThan, LessThanEqual = 0, 1, 2, 3, 4

class Specification:
    @abstractmethod
    def is_satisfied_by(self, candidate):
        pass

    def __and__(self, other):
        return AndSpecification(self, other)

    def __or__(self, other):
        return OrSpecification(self, other)


class AndSpecification(Specification):
    def __init__(self, first: Specification, second: Specification):
        self.first = first
        self.second = second

    def is_satisfied_by(self, candidate):
        return self.first.is_satisfied_by(candidate) and self.second.is_satisfied_by(candidate)

class OrSpecification(Specification):
    def __init__(self, first: Specification, second: Specification):
        self.first = first
        self.second = second

    def is_satisfied_by(self, candidate):
        return self.first.is_satisfied_by(candidate) or self.second.is_satisfied_by(candidate)

class NameFilter(Specification):
    def __init__(self, name):
        self.name = name

    def is_satisfied_by(self, f: File):
        str_len = len(self.name)
        return f.name[0:str_len] == self.name

class SizeFilter(Specification):
    def __init__(self, size, op: Operator):
        self.size = size
        self.op = op

    def is_satisfied_by(self, f: File):
        if self.op == Operator.Equal:
            return f.size == self.size
        elif self.op == Operator.GreaterThan:
            return f.size > self.size
        elif self.op == Operator.GreaterThanEqual:
            return f.size >= self.size
        elif self.op == Operator.LessThan:
            return f.size < self.size
        elif self.op == Operator.LessThanEqual:
            return f.size <= self.size
        return False

class FileSystem(object):
    def __init__(self, root):
        self.curr_directory = root
        self.home = root

    def search(self, search: Specification):
        result = []
        for f in self.curr_directory.children.values():
            if search.is_satisfied_by(f):
                result.append(f.name)
        return result

    def ls(self):
        for f in self.curr_directory.children.values():
            if f.is_file:
                print(f.name)
            else:
                print("/" + f.name)


    def _path(self, path):
        stack = []
        cur = ""
        for c in path + "/":
            if c == "/":
                if cur == "..":
                    if stack:
                        stack.pop()
                elif cur != "" and cur != "." :
                    stack.append(cur)
                cur = ""
            else:
                cur += c
        return stack


    def cd(self,path):
        simplified_path = self._path(path)
        new_directory = self.curr_directory

        if path == "":
            self.curr_directory = self.home

        for path in simplified_path:
            if path in new_directory.children:
                new_directory = new_directory.children[path]
            else:
                print("File don't exist")
                break

        self.curr_directory = new_directory

root = File('/', None, False)
f1 = File('abc.txt', 2, True)
f2 = File('bcd.txt', 5, True)
f3 = File('def.txt', 10, True)
f4 = File('abc.md', 10, True)
sub_root = File('sub_folder_1', None, False)
children = [f1, f2, f3, f4, sub_root]
root.add_children(children)
f5 = File('test1.txt', 3, True)
f6 = File('test2.md', 2, True)
new_folder = File('sub_folder_2',  None, False)
children = [f5,f6,new_folder]
sub_root.add_children(children)
name_filter = NameFilter('abc')
size_filter = SizeFilter(2, Operator.GreaterThanEqual)
search = name_filter.__and__(size_filter)

fs = FileSystem(root)
fs.cd("sub_folder_1")
fs.ls()

test1.txt
test2.md
/sub_folder_2
